### Step 2 - Snapping

In this notebook, we take our origin points (here, derived from an HRSL raster manually using the 'Raster pixels to Points' tool in QGIS, and saved out as shapefile) as the main input. 

At the end of this notebook, we are ready to generate the OD Matrix

In [1]:
import os, sys
import time
import pandas as pd
import geopandas as gpd
import networkx as nx
from shapely.geometry import Point, MultiPoint
from shapely.wkt import loads
from scipy import spatial
from functools import partial
import pyproj
from shapely.ops import transform

import GOSTnets as gn
import geopy

Set path names and file names, and import all the scenario graphs

In [3]:
input_pth = r'inputs\\dests'
interm_pth = r'intermediate\\'
fin_pth = r'final'
origin_folder = r'..\..\..\GEO\Population\CXB'

In [4]:
cur_pckle = r'final_current_G.pickle'
upgr_all = r'final_upgrade_all_G.pickle'
upgr_nosouth = r'final_upgrade_nosouth_G.pickle'
upgr_noferry = r'final_upgrade_noferry_G.pickle'

In [5]:
G_current = nx.read_gpickle(os.path.join(fin_pth, cur_pckle))
G_upgr_all = nx.read_gpickle(os.path.join(fin_pth, upgr_all))
G_upgr_nosouth = nx.read_gpickle(os.path.join(fin_pth, upgr_nosouth))
G_upgr_noferry = nx.read_gpickle(os.path.join(fin_pth, upgr_noferry))

Import origins and destinatins

In [14]:
# Origins
# Note the commented out growth centers: you can use an alternate origin file if you want

# origin_file = 'growth_center_origins.shp'
origin_file = r'hrsl_2018_cxb_pts.shp'

In [7]:
# Destinations

cxb_fil = r'cxb_ctr.shp'
chitt_fil = r'chittagong.shp'
health_fil = r'hc_merge_200324_4326.shp'
primary_fil = r'school_category_primary.gpkg'
secondary_fil = r'school_category_secondary.gpkg'
tertiary_fil = r'school_category_tertiary.gpkg'
matar_fil = r'martarbari.shp'
mkts_fil = r'mkts_merge_4326.shp'
gc_fil = r'cxb_lged_gc_moved_4326.shp'

dests = {"CXB" : cxb_fil, "Chittagong" : chitt_fil, "Health" : health_fil, \
         "Primary_education" : primary_fil, "Secondary_education" : secondary_fil, "Tertiary_education" : tertiary_fil, \
         "Martarbari" : matar_fil, "All_markets" : mkts_fil, "Growth_centers" : gc_fil}


### Origins

Import Origins

In [8]:
origins = gpd.read_file(os.path.join(origin_folder, origin_file))

Snap Origins

In [10]:
# The origins geometry type must be Point, not MultiPoint. Check here.
origins.geom_type

0     Point
1     Point
2     Point
3     Point
4     Point
5     Point
6     Point
7     Point
8     Point
9     Point
10    Point
11    Point
12    Point
13    Point
14    Point
15    Point
16    Point
17    Point
18    Point
19    Point
20    Point
21    Point
22    Point
23    Point
24    Point
25    Point
26    Point
27    Point
28    Point
29    Point
30    Point
31    Point
32    Point
33    Point
dtype: object

In [11]:
origins = gpd.GeoDataFrame(
    origins, geometry=gpd.points_from_xy(origins.centroid.x,origins.centroid.y)) # manually re-calculate as a point geometry

origins.geom_type

0     Point
1     Point
2     Point
3     Point
4     Point
5     Point
6     Point
7     Point
8     Point
9     Point
10    Point
11    Point
12    Point
13    Point
14    Point
15    Point
16    Point
17    Point
18    Point
19    Point
20    Point
21    Point
22    Point
23    Point
24    Point
25    Point
26    Point
27    Point
28    Point
29    Point
30    Point
31    Point
32    Point
33    Point
dtype: object

Clipping origins _exactly_ to CXB extent

In [15]:
shp = gpd.read_file(os.path.join('inputs/cxb_filters/', 'cxb_dist_4326.shp'))
shp = shp.to_crs({'init':'epsg:4326'})
cxb_obj = shp.geometry.iloc[0]

cxb_pop_filter = origins.within(cxb_obj)
origins = origins.loc[cxb_pop_filter]

origins.crs = "EPSG:4326"
len(origins) # CXB polygon clipped

C:\Users\wb467985\.conda\envs\geo\lib\site-packages\pyproj\crs\crs.py:55: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


34

In [16]:
origins_snapped = gn.pandana_snap_c(G_current, 
                                      origins, 
                                      source_crs='epsg:4326',
                                      target_crs='epsg:32646', 
                                      add_dist_to_node_col = True,
                                      time_it=True)

time elapsed for function
1.865009069442749


Save Origins to origin folder

In [17]:
origins_snapped.to_csv(os.path.join(origin_folder, origin_file.replace('.shp','_snapped.csv')))
origins_snapped.to_csv(os.path.join(fin_pth, origin_file.replace('.shp','_snapped.csv')))

### Destinations

First let's take a look at the destinations dict

In [14]:
print(dests)

{'CXB': 'cxb_ctr.shp', 'Chittagong': 'chittagong.shp', 'Health': 'hc_merge_200324_4326.shp', 'Primary_education': 'school_category_primary.gpkg', 'Secondary_education': 'school_category_secondary.gpkg', 'Tertiary_education': 'school_category_tertiary.gpkg', 'Martarbari': 'martarbari.shp', 'All_markets': 'mkts_merge_4326.shp', 'Growth_centers': 'cxb_lged_gc_moved_4326.shp'}


Hiding annoying error messages related to projection formatting

In [15]:
import warnings
warnings.filterwarnings('ignore')

Read in, snap, and export snapped destinations from the *dests* dictionary

In [16]:
for dest, fpth in dests.items():
    
    print(dest)
    
    # Read in the file, convert MultiPoint to Point
    
    gdf = gpd.read_file(os.path.join(input_pth,fpth))
    gdf.to_crs({'init':'epsg:4326'})
    gdf = gpd.GeoDataFrame(
        gdf, geometry=gpd.points_from_xy(gdf.centroid.x, gdf.centroid.y)) # geometries must be manually converted to Point, not MultiPoint
    gdf.crs = "EPSG:4326"
    
    # Pandana snap everything -- this used to take a while, now it doesn't. Thanks Andres!
    
    gdf_snapped = gn.pandana_snap_c(G_current, 
                                     gdf, 
                                     source_crs='epsg:4326',
                                     target_crs='epsg:32646', 
                                     add_dist_to_node_col = True,
                                     time_it=True)
    
    # Export the finished snapped destination to a CSV format for processing in Step 3

    export_name = dest + '_current_snapped.csv'
    gdf_snapped.to_csv(os.path.join(fin_pth, export_name))

CXB
time elapsed for function
2.1539618968963623
Chittagong
time elapsed for function
1.8040006160736084
Health
time elapsed for function
1.9529993534088135
Primary_education
time elapsed for function
2.4129958152770996
Secondary_education
time elapsed for function
1.959000587463379
Tertiary_education
time elapsed for function
2.1689815521240234
Martarbari
time elapsed for function
2.494020700454712
All_markets
time elapsed for function
2.190007209777832
Growth_centers
time elapsed for function
1.9969894886016846


In [17]:
# Upgrade all routine

for dest, fpth in dests.items():
    
    print(dest)
    
    # Read in the file, convert MultiPoint to Point
    
    gdf = gpd.read_file(os.path.join(input_pth,fpth))
    gdf.to_crs({'init':'epsg:4326'})
    gdf = gpd.GeoDataFrame(
        gdf, geometry=gpd.points_from_xy(gdf.centroid.x, gdf.centroid.y)) # geometries must be manually converted to Point, not MultiPoint
    gdf.crs = "EPSG:4326"
    
    # Pandana snap everything -- this will take a while
    
    gdf_snapped = gn.pandana_snap_c(G_upgr_all, 
                                     gdf, 
                                     source_crs='epsg:4326',
                                     target_crs='epsg:32646', 
                                     add_dist_to_node_col = True,
                                     time_it=True)
    
    # Export the finished snapped destination to a CSV format for processing in Step 3

    export_name = dest + '_ua_snapped.csv'
    gdf_snapped.to_csv(os.path.join(fin_pth, export_name))

CXB
time elapsed for function
2.0299999713897705
Chittagong
time elapsed for function
1.770003318786621
Health
time elapsed for function
1.9169912338256836
Primary_education
time elapsed for function
2.271014451980591
Secondary_education
time elapsed for function
1.8190028667449951
Tertiary_education
time elapsed for function
2.0210020542144775
Martarbari
time elapsed for function
2.443016290664673
All_markets
time elapsed for function
2.144007682800293
Growth_centers
time elapsed for function
2.1919937133789062


In [18]:
# Upgrade no southern road routine

for dest, fpth in dests.items():
    
    print(dest)
    
    # Read in the file, convert MultiPoint to Point
    
    gdf = gpd.read_file(os.path.join(input_pth,fpth))
    gdf.to_crs({'init':'epsg:4326'})
    gdf = gpd.GeoDataFrame(
        gdf, geometry=gpd.points_from_xy(gdf.centroid.x, gdf.centroid.y)) # geometries must be manually converted to Point, not MultiPoint
    gdf.crs = "EPSG:4326"
    
    # Pandana snap everything -- this will take a while
    
    gdf_snapped = gn.pandana_snap_c(G_upgr_nosouth, 
                                     gdf, 
                                     source_crs='epsg:4326',
                                     target_crs='epsg:32646', 
                                     add_dist_to_node_col = True,
                                     time_it=True)
    
    # Export the finished snapped destination to a CSV format for processing in Step 3

    export_name = dest + '_uns_snapped.csv'
    gdf_snapped.to_csv(os.path.join(fin_pth, export_name))

CXB
time elapsed for function
2.6109912395477295
Chittagong
time elapsed for function
1.956000566482544
Health
time elapsed for function
2.407012701034546
Primary_education
time elapsed for function
2.687019109725952
Secondary_education
time elapsed for function
2.309990406036377
Tertiary_education
time elapsed for function
2.728011131286621
Martarbari
time elapsed for function
2.2329955101013184
All_markets
time elapsed for function
2.206979513168335
Growth_centers
time elapsed for function
2.391000986099243


In [19]:
# Upgrade no ferry routine

for dest, fpth in dests.items():
    
    print(dest)
    
    # Read in the file, convert MultiPoint to Point
    
    gdf = gpd.read_file(os.path.join(input_pth,fpth))
    gdf.to_crs({'init':'epsg:4326'})
    gdf = gpd.GeoDataFrame(
        gdf, geometry=gpd.points_from_xy(gdf.centroid.x, gdf.centroid.y)) # geometries must be manually converted to Point, not MultiPoint
    gdf.crs = "EPSG:4326"
    
    # Pandana snap everything -- this will take a while
    
    gdf_snapped = gn.pandana_snap_c(G_upgr_noferry, 
                                     gdf, 
                                     source_crs='epsg:4326',
                                     target_crs='epsg:32646', 
                                     add_dist_to_node_col = True,
                                     time_it=True)
    
    # Export the finished snapped destination to a CSV format for processing in Step 3

    export_name = dest + '_unf_snapped.csv'
    gdf_snapped.to_csv(os.path.join(fin_pth, export_name))

CXB
time elapsed for function
2.0620102882385254
Chittagong
time elapsed for function
2.193042516708374
Health
time elapsed for function
2.6059935092926025
Primary_education
time elapsed for function
2.394998073577881
Secondary_education
time elapsed for function
1.9469938278198242
Tertiary_education
time elapsed for function
2.267995834350586
Martarbari
time elapsed for function
1.9496924877166748
All_markets
time elapsed for function
1.9079983234405518
Growth_centers
time elapsed for function
2.0099964141845703
